In [ ]:
#change .xlsx to .txt
# Import the required library (pandas)
import pandas as pd

# Load the excel file
df = pd.read_excel("metazoa.xlsx")

# Extract the first column of the excel
first_column = df[df.columns[0]]

# Write the extracted data to a text file, separating each row with a ";"
with open("output.txt", "w") as file:
    for item in first_column:
        file.write(str(item) + ";")


In [2]:
#General note: 
#Since NCBI Entrez seems to be unstable for the download.
#Two errors often apeared:
#1. breack down of program, it is recomended to restart the code and edit manually before the "output.txt" file
#2. Sometimes e-mail adress in "Entrez.email" is giving an error message, 
#it is recomended to out-comment this part normally it works afterwords




#protein of interest
USH1G="MNDQYHRAARDGYLELLKEATRKELNAPDEDGMTPTLWAAYHGNLESLRLIVSRGGDPDKCDIWGNTPLHLAASNGHLHCLSFLVSFGANIWCLDNDYHTPLDMAAMKGHMECVRYLDSIAAKQSSLNPKLVGKLKDKAFREAERRIRECAKLQRRHHERMERRYRRELAERSDTLSFSSLTSSTLSRRLQHLALGSHLPYSQATLHGTARGKTKMQKKLERRKQGGEGTFKVSEDGRKSARSLSGLQLGSDVMFVRQGTYANPKEWGRAPLRDMFLSDEDSVSRATLAAEPAHSEVSTDSGHDSLFTRPGLGTMVFRRNYLSSGLHGLGREDGGLDGVGAPRGRLQSSPSLDDDSLGSANSLQDRSCGEELPWDELDLGLDEDLEPETSPLETFLASLHMEDFAALLRQEKIDLEALMLCSDLDLRSISVPLGPRKKILGAVRRRRQAMERPPALEDTEL"
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import Entrez, SeqIO
import requests
#Entrez e-mail has to be registered in NCBI and inserted
Entrez.email=example.example@example.com

with open("organism.txt", "r") as f:
    organisms_string = f.read()
organisms_list = organisms_string.split(";")

for i in organisms_list:
    # Specify the organism you want to search for
    organism = i

    # Use the esearch function to search for the organism in the taxonomy database
    result = Entrez.esearch(db="taxonomy", term=organism)
    record = Entrez.read(result)

    # Extract the taxonomy ID of the organism
    taxonomy_id = record["IdList"][0]

    # Use the efetch function to retrieve the nomenclature of the organism
    result = Entrez.efetch(db="taxonomy", id=taxonomy_id, retmode="xml")
    record = Entrez.read(result)

    # Extract the nomenclature of the organism
    nomenclature = record[0]["ScientificName"]


    # Perform BLASTP search on NCBI using USH1G gene as input and organism = homo sapiens (taxonomy id 9606)
    result_handle = NCBIWWW.qblast("blastp", "nr", USH1G, entrez_query="txid"+taxonomy_id+"[Organism]")

    # Parse the BLASTP output
    blast_records = NCBIXML.parse(result_handle)

    # Initialize variables to store the lowest e-value, the highest percent identity and the corresponding record
    lowest_evalue = float("inf")
    highest_identity = 0
    lowest_record = None
    fasta_seq = ""

    # Iterate through each record in the BLASTP output
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                # Check if the current e-value is less than the current lowest e-value
                if hsp.expect < lowest_evalue and (hsp.identities / hsp.align_length)*100 > highest_identity:
                    lowest_evalue = hsp.expect
                    highest_identity = (hsp.identities / hsp.align_length)*100
                    lowest_record = hsp
                    title_fasta_pre=alignment.title
                    #print(title_fasta_pre)
                    title_fasta=title_fasta_pre.split(";")[0]
            
                    title_fasta_2=title_fasta.split("|")[1]
                    #print(title_fasta_2)
                    
    handle = Entrez.efetch(db="protein", id=title_fasta_2, rettype="fasta")
    record = SeqIO.read(handle, "fasta")

    fasta_code = str(record.seq)
                    
                

    if lowest_record is not None:
            # Extract the FASTA sequence of the alignment
            fasta_seq = ">" + title_fasta_2 + "\n" + fasta_code + "\n"
            # Create the output file name by combining the taxonomy ID and fasta extension
            output_file = taxonomy_id+".fasta"
            # Write the FASTA sequence to a file
            with open(output_file, "w") as f:
                f.write(fasta_seq)
    else:
        print("No records found.")

NameError: name 'example' is not defined

In [ ]:
#merges all .fasta into one .txt

import os

def merge_fasta_files():
    # Get the list of all '.fasta' files in the current directory
    fasta_files = [file for file in os.listdir() if file.endswith('.fasta')]

    # Create a new file to write the contents of all '.fasta' files
    with open('fasta_merged.txt', 'w') as out_file:
        for file in fasta_files:
            with open(file, 'r') as fasta:
                out_file.write(fasta.read())

if __name__ == '__main__':
    merge_fasta_files()